In [1]:
1+1

2

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder 
import pickle

In [27]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [28]:
## Preprocess the data,drop columns that are not needed
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [29]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [30]:
## Encode categorical data

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [31]:
## One hot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder_geo = OneHotEncoder()
geo_encoder = oneHotEncoder_geo.fit_transform(data[['Geography']])
geo_encoder


<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [32]:
oneHotEncoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [33]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=oneHotEncoder_geo.get_feature_names_out(['Geography'])) 
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [34]:
## combine columns
data = pd.concat([data,geo_encoded_df],axis=1)  
data = data.drop(['Geography'],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [35]:
## save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender ,f)

with open('oneHotEncoder_geo.pkl','wb') as f:
    pickle.dump(oneHotEncoder_geo ,f) 

In [36]:
## Divide the dataset into independant & dependant features
X = data.drop(['Exited'],axis=1)
y=data['Exited']

## Split the dataset into training and testing
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 

In [37]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [38]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler ,f)

ANN Implementation

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [40]:
(X_train.shape[1],)

(12,)

In [44]:
## Build ANN model
model = Sequential([
  Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## Hideen layer 1
  Dense(32,activation='relu'), ## Hidden layer 2
  Dense(1,activation='sigmoid') ## Output layer
]
)


In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [47]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [49]:
## setup Tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [52]:
## setup Early STopping
early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [53]:
## Train the model
history = model.fit(
  X_train,y_train,validation_data=(X_test,y_test),epochs=100,
  callbacks=[early_stopping,tensorflow_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3371 - accuracy: 0.8601 - val_loss: 0.3485 - val_accuracy: 0.8610
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3345 - accuracy: 0.8633 - val_loss: 0.3418 - val_accuracy: 0.8595
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3315 - accuracy: 0.8650 - val_loss: 0.3527 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3302 - accuracy: 0.8652 - val_loss: 0.3406 - val_accuracy: 0.8615
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3289 - accuracy: 0.8643 - val_loss: 0.3482 - val_accuracy: 0.8540
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3246 - accuracy: 0.8671 - val_loss: 0.3451 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3248 - accuracy: 0.8637 - val_loss: 0.3484 - val_accuracy: 0.8640

In [54]:
model.save('churn_model.h5')

d:\misc\code\GenAI\ANNClass\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [57]:
## Load Tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [58]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16964), started 0:42:19 ago. (Use '!kill 16964' to kill it.)